# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察填補缺值以及 標準化 / 最小最大化 對數值的影響

# [作業重點]
- 觀察替換不同補缺方式, 對於特徵的影響 (In[4]~In[6], Out[4]~Out[6])
- 觀察替換不同特徵縮放方式, 對於特徵的影響 (In[7]~In[8], Out[7]~Out[8])

In [23]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

# 讀取訓練與測試資料
data_path = 'data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

# 重組資料成為訓練 / 預測用格式
train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1) #axis=1 column #axis=0 row
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test]) #concat() 合併兩個或多個陣列
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [24]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns): ##zip & for 將兩個 list 以迴圈的方式一次各取一個元素出來處理
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

5 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



In [25]:
# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
train_num = train_Y.shape[0] #891 #row
df.head()

891


,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500


# 作業1
* 試著在補空值區塊, 替換並執行兩種以上填補的缺值, 看看何者比較好?

In [26]:
"""
Your Code Here
"""
# 空值補 -1, 做羅吉斯(邏輯 Logistic)迴歸：分類問題 建立「二元目標變數」(Binary Output Variable) 跟解釋變數之間的關係 模型形式：條件機率=乙狀函數
df_m1 = df.fillna(-1)
train_X = df_m1[:train_num]
estimator = LogisticRegression(solver='liblinear') #solver 用於優化問題的算法 #liblinear 適合用小型數據集
#使用邏輯回歸提示 FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.FutureWarning)
#根據報警信息和參考相關文檔，“ Default will change from 'liblinear' to 'lbfgs' in 0.22. ”，默認的solver參數在0.22版本中，將會由“liblinear”變為“lbfgs”，且指定solver參數可以消除該warning。
   
cross_val_score(estimator, train_X, train_Y, cv=5).mean()
#cross_val_score() 通過交叉驗證評估分數
# estimator 實現“ fit”的估計器對象 用於擬合數據的對象。
# X ： 類似數組要擬合的數據。例如可以是列表或數組。
# y ： 類似數組，可選，默認值：無  在監督學習的情況下要嘗試預測的目標變量。
#cv ： int，交叉驗證生成器或可迭代的  無，要使用默認的三折交叉驗證  整數，用於指定的折疊次數(Stratified)KFold，

0.6960299128976762

In [27]:
# 空值補 0
df_0 = df.fillna(0)
train_X = df_0[:train_num]
estimator = LogisticRegression(solver='liblinear')
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6971535084032942

In [28]:
# 空值補平均值
df_mn = df.fillna(df.mean())
train_X = df_mn[:train_num]
estimator = LogisticRegression(solver='liblinear')
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6981761033723469

# 作業2
* 使用不同的標準化方式 ( 原值 / 最小最大化 / 標準化 )，搭配羅吉斯迴歸模型，何者效果最好?

In [29]:
"""
Your Code Here
"""
# 空值補 -1, 搭配最大最小化
df = df.fillna(-1)
df_temp = MinMaxScaler().fit_transform(df)
#fit_transform()先擬合數據，然後轉化它將其轉化為標準形式，比如降維、特徵提取、標準化
# MinMaxScaler() 將屬性縮放到一個指定的最大和最小值（通常是1-0）之間
# 目的包括： 1、對於方差非常小的屬性可以增強其穩定性。 2、維持稀疏矩陣中為0的條目。
train_X = df_temp[:train_num]
estimator = LogisticRegression(solver='liblinear')
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6971346062663598

In [30]:
# 搭配標準化
df_temp = StandardScaler().fit_transform(df)
#作用：去均值和方差歸一化。且是針對每一個特徵維度來做的，而不是針對樣本。
train_X = df_temp[:train_num]
estimator = LogisticRegression(solver='liblinear')
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6982582017719778